In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import math
import matplotlib.pyplot as plt

path = 'Raw_wearable_data/'
path_class = 'Class_wearable_data1/'
path_classtable = 'Participant_class_info/'

In [ ]:
# Read class table and participant information
class_time = pd.read_csv(path_classtable + 'Class_table.csv')
class_room = pd.read_csv(path_classtable + 'Student.csv')
teacher_time = pd.read_csv(path_classtable + 'Teacher.csv')
class_room.set_index('Pid', inplace = True)

# Update data types

In [ ]:
# Update the dataframe with timestamps
def convert_time(data):
    Sampling = data.iloc[1,0]
    data['Date'] = data.iloc[0,0]
    data = data[2:]    
    Time = np.arange(0, len(data)/Sampling, 1/Sampling)
    data['Time'] = Time.tolist()
    data.Date = data.Date + data.Time
    data.Date = pd.to_datetime(data['Date'], unit='s')
    data.Date = data.Date + timedelta(hours=10)
    data.reset_index(inplace=True, drop=True)
    return data

In [ ]:
# Read the date folders save the name of each folder into a list
date = []
entries = os.scandir(path)
for i in entries:
    date.append(i.name)

In [ ]:
# Create a temporary folder to save the processed data
path1 = 'Processed_wearable_data/'
if not os.path.exists(path1):
    os.makedirs(path1)

In [ ]:
# Process the wearable data save it into the temporary folder
def Create_folder(z):
    if os.path.isfile(path1 + i + '/' + pid + '/' + fi.name):
        df.to_csv(path1 + i + '/' + pid + '/' + fi.name, index=False, mode='a', header=False)
    else:
        df.to_csv(path1 + i + '/' + pid + '/' + fi.name, index=False)
warnings.filterwarnings('ignore')        
for i in date:
    print('-----------------'+i+'-----------------')
    
    # Read all records in each day
    bands = []
    band_entry = os.scandir(path + i)
    
    # Create the list containing all bands names
    for j in band_entry:
        if j.name[-3:] == 'zip':
            pass
        else:
            bands.append(j.name)
    
    # Going Through each file in the bands file
    for z in bands:
        print(z)
        files = os.scandir(path + i + '/' + z)
        pid = z[z.find('_')+1:]
        os.makedirs(path1 + i + '/' + pid, mode=0o770, exist_ok= True)
        for fi in files:
            if fi.name == 'info.text':
                pass
            
            elif fi.name == 'tags.csv':
                if os.stat(path + i + '/' + z + '/' + fi.name).st_size == 0:
                    pass
                else:
                    df = pd.read_csv(path + i + '/' + z + '/' + fi.name, header=None)
                    df[0] = pd.to_datetime(df[0], unit = 's') + timedelta(hours=10)
                    df.rename(columns={0:'Tags'}, inplace=True)
                    Create_folder(z)

            elif fi.name == 'IBI.csv':
                if os.stat(path + i + '/' + z + '/' + fi.name).st_size == 0:
                    pass
                else:
                    df = pd.read_csv(path + i + '/' + z + '/' + fi.name, header=None)
                    ibi_prim = df[1:]
                    ibi_prim[0] = ibi_prim[0] + df[0][0]
                    ibi_prim[0] = pd.to_datetime(ibi_prim[0], unit = 's') + timedelta(hours=10)
                    ibi_prim.rename(columns={0:'Time', 1:'IBI'}, inplace=True)
                    if os.path.isfile(path1 + i + '/' + pid + '/' + fi.name):
                        ibi_prim.to_csv(path1 + i + '/' + pid + '/' + fi.name, index=False, mode='a', header=False)
                    else:
                        ibi_prim.to_csv(path1 + i + '/' + pid + '/' + fi.name, index=False)

            elif fi.name =='EDA.csv' :
                df = pd.read_csv(path + i + '/' + z + '/' + fi.name, header=None)
                df.rename(columns={0:'EDA'}, inplace=True)
                df = convert_time(df)
                Create_folder(z)
                
            elif fi.name == 'ACC.csv':
                df = pd.read_csv(path + i + '/' + z + '/' + fi.name, header=None)
                df.rename(columns = {0:'X',1:'Y',2:'Z'}, inplace=True)
                df = convert_time(df)
                Create_folder(z)

            elif fi.name == 'BVP.csv':
                df = pd.read_csv(path + i + '/' + z + '/' + fi.name, header=None)
                df.rename(columns = {0:'BVP'}, inplace=True)
                df = convert_time(df)
                Create_folder(z)

            elif fi.name == 'HR.csv':
                df = pd.read_csv(path + i + '/' + z + '/' + fi.name, header=None)
                df.rename(columns={0:'HR'}, inplace=True)
                df = convert_time(df)
                Create_folder(z)
                        
            elif fi.name == 'TEMP.csv':
                df = pd.read_csv(path + i + '/' + z + '/' + fi.name, header=None)
                df.rename(columns = {0:'TEMP'}, inplace = True)
                df = convert_time(df)
                Create_folder(z)

# Class division

In [ ]:
# Function that compute the class allocation
def get_class_room(name, form, math, language, science):
    if name == 'Maths':
        return math
    elif name == 'Language':
        return language
    elif name == 'Science':
        return science
    else:
        return form
def get_class_info(id, date):
    classes = pd.DataFrame()
    form_r, math_r, language_r = class_room.loc[id][['Form Room', 'Math Room', 'Language Room']]
    arrange = class_time[class_time['Date'] == int(date)]
    for rowid,row in arrange.iterrows():
        room = row['Room']
        subject = row['Subject']
        if subject == 'Language' and room == language_r:
            classes = classes.append(row)
        elif subject == 'Maths' and room == math_r:
            classes = classes.append(row)
        elif subject == 'Assembly' or subject == 'Chapel':
            classes = classes.append(row)
        elif subject in ['Form', 'English', 'Science', 'Politics', 'PE', 'Health']:
            if row['is_Form'] == form_r:
                classes = classes.append(row)
    classes['Pid'] = id
    classes[['Pid','Class_id','Class_no','Date','Week','Weekday']] = classes[
        ['Pid','Class_id','Class_no','Date','Week','Weekday']].astype('int')
    return classes

In [ ]:
# Save all folders with different date into a list
Date = []
entries = os.scandir(path1)
for i in entries:
    if len(i.name) == 8:
        Date.append(i.name)

In [ ]:
# Divide classes
for date in Date:
    bands = []
    id_bands = os.scandir(path1 + date)
    print('------------------Date:' + date + '----------------')
    for pid in id_bands:
        try:
            bands.append(pid.name)
        except:
            pass
    for pid in bands:
        print('*********' + 'Student id:' + str(pid))
        
        # get the time slot for each class
        mask = get_class_info(int(pid),date)
        
        # open the files for student 'pid' on the day 'date'
        files = os.scandir(path1 + date +'/'+str(pid))
        
        for record in files:
            if pd.read_csv(path1 + date + '/' + str(pid) + '/' + record.name).empty:
                print(record.name +' is empty.')
            elif record.name == 'tags.csv':
                pass
            elif record.name == '.ipynb_checkpoints':
                pass
            else:
                df = pd.read_csv(path1 + date + '/' + str(pid) + '/' + record.name)
                print(date + ' ' + str(pid) +' ' + record.name)
                if record.name == 'IBI.csv':
                    df['Date'] = pd.to_datetime(df.Time).apply(datetime.time)
                else:
                    df.Date = pd.to_datetime(df.Date).apply(datetime.time)
                for w in range(len(mask)):
                    class_1 = df[(pd.to_datetime(mask.iloc[w]['Start_time']).time() < df['Date']) &
                                (df['Date'] < pd.to_datetime(mask.iloc[w]['Finish_time']).time())]
                    if class_1.empty:
                        pass
                    else:
                        save_folder = str(mask.iloc[w]['Class_id'])
                        save_file = str(mask.iloc[w]['Pid'])
                        os.makedirs(path_class + save_folder + '/' + save_file, exist_ok=True)
                        class_1.to_csv(path_class + save_folder + '/' + save_file + '/' + record.name, index=False)
                        print('Class: ' + save_folder + '_' + save_file + ' ' + record.name)      